> conv提特征 向量点积往大收敛 提取的特证逐渐与原图相似

In [1]:
# demo
import torch
import torch.nn as nn
import torch.nn.functional as F
in_channels = 1
out_channels = 1
kernel_size = 3 #传入元组 卷积核可以为矩形
batch_size = 2
bias = False
input_size = [batch_size, in_channels, 4, 4]
# 实例化类 首字母大写
conv_layer_test = torch.nn.Conv2d(3, 9, kernel_size, bias=bias)
# padding = sample
conv_layer = torch.nn.Conv2d(1, 1, kernel_size, bias=bias)
print(conv_layer.weight)
x = torch.randn(input_size)
x = F.dropout(x)
x = conv_layer(x)
print(conv_layer_test.weight.shape)

# print(x)

Parameter containing:
tensor([[[[-0.3321, -0.1891,  0.1154],
          [ 0.2970,  0.2722,  0.1685],
          [ 0.0735, -0.1254, -0.1498]]]], requires_grad=True)
torch.Size([9, 3, 3, 3])


> 函数实现

In [2]:
# 用原始的矩阵运算实现二维卷积,先不考虑batchsize维度和channel维数
def matrix_multiplication_for_conv2d(input, kernel, stride=1):
    input_h, input_w = input.shape

    return